In [1]:
import os


In [2]:
%pwd

'c:\\Users\\91888\\Desktop\\code\\Text-Summarization-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\91888\\Desktop\\code\\Text-Summarization-Project'

In [5]:
from dataclasses import dataclass
from pathlib import  Path

@dataclass(frozen=True)
class DataIngestionConfig: 
    root_dir:Path
    source_URL: str 
    local_data_file: Path 
    unzip_dir: Path   

In [1]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath= CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH:

        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
    )

    create_directories([self.config.artifacts_root])